# pip install 
- pip install pandas
- pip install numpy
- pip install yfinance
https://pypi.org/project/yfinance/
- pip install finviz
https://pypi.org/project/finviz/
- pip install finvizfinance
https://pypi.org/project/finvizfinance/

# Import required libs
- import yfinance as yf
- from finviz.screener import Screener
- import pandas as pd 
- import numpy as np
- from tqdm import tqdm
- from scipy.stats import linregress
- from IPython.display import Image ,display
- from IPython.core.display import HTML
- from finvizfinance.quote import finvizfinance

# ignore warning caused by python version
- warnings.filterwarnings('ignore')

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import yfinance as yf
from finviz.screener import Screener
import pandas as pd 
import numpy as np
from tqdm import tqdm
from scipy.stats import linregress
from IPython.display import Image ,display
from IPython.core.display import HTML
from finvizfinance.quote import finvizfinance

# retrieve data from finviz
- filters = ['cap_midover','sh_curvol_o50','sh_price_o10','fa_epsqoq_o5','fa_salesqoq_o5','sh_instown_o10','ta_highlow52w_b0to10h']
- stock_list = Screener(filters = filters , table = 'Performance' , order = 'Price')
- stock_list.data


In [3]:
filters = ['cap_midover','sh_curvol_o50','sh_price_o10','fa_epsqoq_o5','fa_salesqoq_o5','sh_instown_o10','ta_highlow52w_b0to10h']
stock_list = Screener(filters = filters , table = 'Performance' , order = 'Price')

#https://finviz.com/screener.ashx?v=111&f=cap_midover,fa_epsqoq_o5,fa_salesqoq_o5,sh_curvol_o50,sh_instown_o10,sh_price_o10&ft=3
stock_list

stock_list.data

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.17s/it]


[{'No.': '1',
  'Ticker': 'CVE',
  'Perf Week': '5.12%',
  'Perf Month': '20.91%',
  'Perf Quart': '3.50%',
  'Perf Half': '32.54%',
  'Perf Year': '175.62%',
  'Perf YTD': '66.56%',
  'Volatility W': '3.72%',
  'Volatility M': '3.88%',
  'Recom': '2.50',
  'Avg Volume': '8.63M',
  'Rel Volume': '1.48',
  'Price': '10.38',
  'Change': '3.22%',
  'Volume': '947,153'},
 {'No.': '2',
  'Ticker': 'NRZ',
  'Perf Week': '-1.26%',
  'Perf Month': '-1.26%',
  'Perf Quart': '4.49%',
  'Perf Half': '-1.53%',
  'Perf Year': '34.61%',
  'Perf YTD': '9.96%',
  'Volatility W': '1.75%',
  'Volatility M': '1.60%',
  'Recom': '1.70',
  'Avg Volume': '5.31M',
  'Rel Volume': '0.56',
  'Price': '10.93',
  'Change': '0.00%',
  'Volume': '221,455'},
 {'No.': '3',
  'Ticker': 'AMCR',
  'Perf Week': '-0.85%',
  'Perf Month': '-7.88%',
  'Perf Quart': '1.30%',
  'Perf Half': '-1.68%',
  'Perf Year': '6.18%',
  'Perf YTD': '-0.68%',
  'Volatility W': '1.93%',
  'Volatility M': '1.70%',
  'Recom': '3.00',
  'Av

# retrieve Ticker data

In [4]:
ticker = yf.Ticker('AMCR')
ticker_history = ticker.history(period = 'max')

ticker_history

ticker_table =pd.DataFrame(stock_list.data)
ticker_table

ticker_list = ticker_table['Ticker'].to_list()
ticker_list

for ticker_string in tqdm(ticker_list):
    ticker = yf.Ticker(ticker_string)
    ticker_history = ticker.history(period = 'max')


100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [01:28<00:00,  1.37it/s]


# define methods

In [11]:

def slope_reg(arr):
    y = np.array(arr)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    return slope


def stock_filter(df):
    df['SMA_50'] = df['Close'].rolling(window =50).mean()
    df['SMA_150'] = df['Close'].rolling(window =150).mean()
    df['SMA_200'] = df['Close'].rolling(window =200).mean()
    df['SMA_slope_200'] = df['SMA_200'].rolling(window =20).apply(slope_reg)
    df['52_week_low'] = df['Close'].rolling(window =5*52).min()
    df['52_week_high'] = df['Close'].rolling(window =5*52).max()
    
    #Criteria 1 :The current price of the security must be greater than 150 and 200-day simple moving averages.
    df['Criteria1'] =(df['Close'] > df['SMA_150']) & (df['Close'] > df['SMA_200'])
    
    #Criteria 2 : The 150-day simple moving average must be greater than the 200-day simple moving average.
    df['Criteria2'] =(df['SMA_150'] > df['SMA_200'])
    
    #Criteria 3: The 200-day simple moving average must be trending up for at least 1 month.
    df['Criteria3'] = df['SMA_slope_200'] > 0.0
    
    #Criteria 4: The 50-day simple moving average must be greater than the 150 simple moving average and the 200 SMA.
    df['Criteria4'] = ( df['SMA_50'] > df['SMA_150']) & ( df['SMA_150'] > df['SMA_200'])
    
    #Criteria 5: The current price must be greater than the 50-day simple moving average.
    df['Criteria5'] = ( df['Close'] > df['SMA_50'])
    
    #Criteria 6: The current price must be at least 30% above the 52 week low.
    df['Criteria6'] = ( df['Close'] - df['52_week_low'])/df['52_week_low'] > 0.3
    
    #Criteria 7: The current price must be within 15% of the 52 week high.
    df['Criteria7'] = ((df['Close'] - df['52_week_high'])/df['52_week_high'] < 0.15) & ((df['Close'] - df['52_week_high'])/df['52_week_high'] > -0.15)
    
    #Criteria 8: replace RS rating
    df['Criteria8'] = (df['Close'] - df['Close'].shift(periods = 250))/df['Close'].shift(periods = 250) > 0.89
    
    #Criteria 9: Pivot (5day)breakout
    df['Criteria9'] = df['Close'] > ((df['Close']).rolling(window= 5).mean() +( df['Close']).rolling(window=5).max() +(df['Close']).rolling(window= 5).min())/3
    
    #Criteria 10: last contraction below 10% 
    df['Criteria10'] = ((df['Close']).rolling(window =10).max() - (df['Close']).rolling(window =10).min()) / (df['Close'].rolling(window =10).min()) < 0.1
                                                                                                                                                                                                                
    df['Fulfilment'] = df[['Criteria1','Criteria2','Criteria3','Criteria4','Criteria5','Criteria6','Criteria7','Criteria8','Criteria9','Criteria10']].all(axis='columns')
                                                                                                            
    return df[['Close','Fulfilment']]




def show_image(ticker):
    stock = finvizfinance(ticker)
    print(stock.TickerCharts())
    display(Image(url= stock.TickerCharts()))


def test_vcp(df):
    df['Future_Close_1'] = df['Close'].shift(periods = -1)
    df['Future_Close_3'] = df['Close'].shift(periods = -3)
    df['Future_Close_5'] = df['Close'].shift(periods = -5)
    df['Future_Close_7'] = df['Close'].shift(periods = -7)
    df['Future_Close_9'] = df['Close'].shift(periods = -9)
    df['Result_1'] = ( df['Future_Close_3'] - df['Close']) / df['Close']
    df['Result_3'] = ( df['Future_Close_5'] - df['Close']) / df['Close']
    df['Result_5'] = ( df['Future_Close_7'] - df['Close']) / df['Close']
    df['Result_7'] = ( df['Future_Close_9'] - df['Close']) / df['Close']
    vcp_date = df[df['Fulfilment'] == True]
    print(vcp_date[['Result_1','Result_3','Result_5','Result_7']])
    print(vcp_date[['Result_1','Result_3','Result_5','Result_7']].describe())                



# find out and backtest VCP models

In [12]:
ticker = yf.Ticker("CRVL")
ticker_history = ticker.history(period = 'max')
data = stock_filter(ticker_history)
test_vcp(data)


for ticker_string in tqdm(ticker_list):
    ticker = yf.Ticker(ticker_string)
    ticker_history = ticker.history(period = 'max')
    data = stock_filter(ticker_history)
    if data['Fulfilment'].tail(1).iloc[0] == True:
        print(ticker_string)
        print(data)
        show_image(ticker_string)
        test_vcp(data)



  0%|                                                                                          | 0/121 [00:00<?, ?it/s]

            Result_1  Result_3  Result_5  Result_7
Date                                              
1993-10-12  0.000000  0.010753 -0.010752 -0.107527
1993-10-15  0.021506 -0.043011 -0.161290 -0.096774
1993-10-19 -0.053191 -0.170213 -0.106383 -0.106383
1993-10-20 -0.126316 -0.178947 -0.094737 -0.094737
1994-03-18  0.000000 -0.113207 -0.160377 -0.132075
...              ...       ...       ...       ...
2021-09-23 -0.016567 -0.001608  0.021177       NaN
2021-09-29  0.020029       NaN       NaN       NaN
2021-09-30       NaN       NaN       NaN       NaN
2021-10-01       NaN       NaN       NaN       NaN
2021-10-04       NaN       NaN       NaN       NaN

[127 rows x 4 columns]
         Result_1    Result_3    Result_5    Result_7
count  124.000000  123.000000  123.000000  121.000000
mean     0.010844    0.015745    0.018338    0.021223
std      0.046841    0.064271    0.068462    0.075030
min     -0.126316   -0.178947   -0.179245   -0.197229
25%     -0.014005   -0.010707   -0.017679  

  3%|██▋                                                                               | 4/121 [00:06<03:05,  1.59s/it]

CNHI
                Close  Fulfilment
Date                             
2013-09-30  11.062305       False
2013-10-01  11.434001       False
2013-10-02  11.416300       False
2013-10-03  11.159656       False
2013-10-04  11.186204       False
...               ...         ...
2021-09-28  16.959999       False
2021-09-29  16.860001       False
2021-09-30  16.610001       False
2021-10-01  16.820000       False
2021-10-04  16.795000        True

[2018 rows x 2 columns]
https://finviz.com/chart.ashx?t=CNHI&ty=c&ta=1&p=d


  4%|███▍                                                                              | 5/121 [00:10<04:48,  2.49s/it]

...

 69%|████████████████████████████████████████████████████████▏                        | 84/121 [04:39<02:11,  3.57s/it]

SPG
                 Close  Fulfilment
Date                              
1993-12-14    5.045592       False
1993-12-15    5.045592       False
1993-12-16    5.045592       False
1993-12-17    5.017557       False
1993-12-20    5.045592       False
...                ...         ...
2021-09-28  133.710007       False
2021-09-29  133.850006       False
2021-09-30  129.970001       False
2021-10-01  133.139999        True
2021-10-04  132.619995        True

[7002 rows x 2 columns]
https://finviz.com/chart.ashx?t=SPG&ty=c&ta=1&p=d


 70%|████████████████████████████████████████████████████████▉                        | 85/121 [04:46<02:44,  4.57s/it]

            Result_1  Result_3  Result_5  Result_7
Date                                              
2010-02-26 -0.010601  0.014561  0.032316  0.033849
2010-03-01 -0.010077  0.017730  0.030230  0.046939
2010-03-05  0.016870  0.033363  0.059801  0.064585
2010-03-08  0.014413  0.026194  0.073443  0.046999
2010-03-09  0.015590  0.041574  0.046276  0.051349
...              ...       ...       ...       ...
2021-09-23 -0.010435 -0.038114 -0.018502       NaN
2021-09-24  0.000000 -0.005304       NaN       NaN
2021-09-27 -0.031159 -0.011405       NaN       NaN
2021-10-01       NaN       NaN       NaN       NaN
2021-10-04       NaN       NaN       NaN       NaN

[94 rows x 4 columns]
        Result_1   Result_3   Result_5   Result_7
count  92.000000  92.000000  90.000000  88.000000
mean   -0.002144   0.002420   0.006174   0.005988
std     0.027628   0.033114   0.037701   0.041127
min    -0.085148  -0.100407  -0.099200  -0.091868
25%    -0.016072  -0.014104  -0.016017  -0.022434
50%    -0.0007

100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [06:56<00:00,  3.44s/it]
